# Evaluate Gender Synonyms

## Imports

In [1]:
import torch
import clip
import numpy as np
import pandas as pd
import json

## Aux Classes

In [2]:
def model_setup(model):
    """Initial loading of CLIP model."""

    available_models = clip.available_models()

    if model in available_models:
        print(f'Loading model: {model}')
        chosen_model = model
    else:
        print(f'{model} unavailable! Using default model: ViT-L/14@336px')
        chosen_model = available_models[0]

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    model, pps = clip.load(chosen_model, device=device, jit=False)

    print(f'Done! Model loaded to {device} device')
    return model, pps


def get_similarities(img, txts):
    """Grab similarity between text and image embeddings."""
    image_features = torch.from_numpy(img).to('cuda')
    similarity = 100.0 * image_features @ txts.T

    return similarity


def get_sims_dict(img_embs, txt_prompts, txt_embs):
    """Generate dictionary with filename and similarities
    scores between text prompts"""
    final_dict = {}
    for _, emb in img_embs.iterrows():
        name = emb['file']
        img_features = emb['embeddings']
        img_sims = get_similarities(img_features, txt_embs)
        sims_dict = {}
        for label, score in zip(txt_prompts, img_sims[0]):
            sims_dict[label] = score.cpu().numpy().item()
        final_dict[name] = sims_dict
    return final_dict


def get_top_synm(final_dict):
    """Grab most similar synonym"""
    files = final_dict.keys()
    wins = []
    for val in final_dict.values():
        scores_list = list(val.values())
        label_list = list(val.keys())
        np_scores = np.asarray(scores_list)
        windex = np.where(np_scores == np_scores.max())[0][0]
        wins.append(label_list[windex])

    top_synm_dict = {'file': files, 'winner': wins}
    return top_synm_dict


def get_sum_synms(final_dict, man_prompts):
    """Ensemble over avg sum of similarities
    between male and female synms"""
    files = final_dict.keys()
    preds = []

    for key, val in final_dict.items():
        man_score = 0
        woman_score = 0
        for k, v in val.items():
            if k in man_prompts:
                man_score += v
            else:
                woman_score += v
        preds.append('Male' if man_score > woman_score else 'Female')

    sum_dict = {'file': files, 'preds': preds}
    return sum_dict


def synm_to_gender(synm, man_prompts):
    """Mapper function to eval between Male and Female
    synonyms"""
    if synm in man_prompts:
        return 'Male'
    else:
        return 'Female'


def generate_final_df(fface_df, score_df):
    """Join the winning class df with the original df"""
    new_df = fface_df.set_index(
        'file').join(score_df.set_index('file'))
    return new_df


def map_synm_to_gender(df, man_prompts):
    """Use sub-set of man synms to evaluate and map
    synms to Male or Female"""
    new_df = df.copy()
    new_df['winner'] = df['winner'].map(lambda x: synm_to_gender(x, man_prompts))
    return new_df


## Load Variables

In [3]:
ROOT = "/home/lazye/Documents/ufrgs/mcs/clip/clip-bias-explore/\
fair-face-classification"
LABELS_PATH = ROOT + "/data/labels"
EMBS_PATH = ROOT + "/data/embeddings"

fface_df = pd.read_csv(ROOT+"/data/fface_val.csv")
vit_model = model_setup('ViT-B/16')
img_embs = pd.read_pickle(EMBS_PATH+"/fface_val_img_embs.pkl")
txt_embs = torch.load(EMBS_PATH+"/synms_labels.pt")

with open(LABELS_PATH+"/bots_synms_prompts.json") as f:
    data = json.load(f)

prompt_list = list(data.values())
man_prompts = prompt_list[:10]
woman_prompts = prompt_list[10:]

Loading model: ViT-B/16
Done! Model loaded to cuda device


In [4]:
sims_dict = get_sims_dict(img_embs, prompt_list, txt_embs)

In [5]:
sims_dict

{'val/1.jpg': {'a photo of a young man': 23.609375,
  'a photo of an adult male': 22.28125,
  'a photo of a male person': 24.578125,
  'a photo of a man': 22.546875,
  'a photo of a guy': 24.09375,
  'a photo of a boy': 26.90625,
  'a photo of a middle-aged man': 22.765625,
  'a photo of an old man': 22.40625,
  'a photo of a grandfather': 22.671875,
  'a photo of a grandpa': 21.703125,
  'a photo of a young woman': 18.34375,
  'a photo of an adult female': 19.21875,
  'a photo of a female person': 22.078125,
  'a photo of a woman': 19.421875,
  'a photo of a lady': 19.0,
  'a photo of a girl': 22.25,
  'a photo of a madam': 21.78125,
  'a photo of an old woman': 20.5625,
  'a photo of a grandmother': 21.8125,
  'a photo of a grandma': 21.859375},
 'val/2.jpg': {'a photo of a young man': 20.71875,
  'a photo of an adult male': 22.40625,
  'a photo of a male person': 23.78125,
  'a photo of a man': 22.625,
  'a photo of a guy': 23.15625,
  'a photo of a boy': 22.09375,
  'a photo of a m

In [6]:
import json

with open('similarities.json', 'w') as f:
    json.dump(sims_dict, f)